incell, outcell座標は別々に計算．それぞれが独立を目指す

In [1]:
import numpy as np

dia_prod = 30.0
thk_i2o = 1.2
ratio_slit = 3
pitch_x = 5.5
pitch_y = 6.0
pitch_slit = 10.0
lim_slit = 13.0

In [2]:
def ref_coords(dia_prod: float, pitch_x: float) -> np.ndarray:
    """基準となるX方向のセル座標を生成.

    Parameters
    ----------
    dia_prod : float
        _description_
    pitch_x : float
        _description_

    Returns
    -------
    np.ndarray
        Shape = (N, 2)
    """
    ref_length_x = 0.5 * dia_prod + 2.0 * pitch_x
    cnt = np.int64(np.ceil((ref_length_x / pitch_x)))
    num = np.arange(-cnt, cnt + 1, 1)
    coords_x = num * pitch_x
    return np.column_stack((coords_x, np.zeros_like(coords_x)))


oc = ref_coords(dia_prod, pitch_x)
print(oc.shape)
print(oc)

(11, 2)
[[-27.5   0. ]
 [-22.    0. ]
 [-16.5   0. ]
 [-11.    0. ]
 [ -5.5   0. ]
 [  0.    0. ]
 [  5.5   0. ]
 [ 11.    0. ]
 [ 16.5   0. ]
 [ 22.    0. ]
 [ 27.5   0. ]]


In [3]:
def init_incell(
    dia_prod: float,
    thk_i2o: float,
    ratio_slit: int,
    pitch_x: float,
    pitch_y: float,
) -> np.ndarray:
    """インセルの初期配置を生成.

    Parameters
    ----------
    dia_prod : float
        _description_
    thk_i2o : float
        _description_
    ratio_slit : int
        _description_
    pitch_x : float
        _description_
    pitch_y : float
        _description_

    Returns
    -------
    np.ndarray
        Shape = (N, 2)
    """
    # 基準座標:Shape(X個数, XY座標)
    ref = ref_coords(dia_prod, pitch_x)
    # 列数分複製:Shape(Y個数, X個数, XY座標)
    res = np.tile(ref, (ratio_slit, 1, 1))

    # オフセット量:Shape(Y個数)
    offset_y = np.arange(0, pitch_y * ratio_slit, pitch_y)
    offset_y += thk_i2o

    # オフセット行列:Shape(Y個数, X個数, XY座標)
    offset_mat = np.zeros(res.shape)
    offset_mat[::2, :, 0] += 0.5 * pitch_x
    offset_mat[:, :, 1] += offset_y[:, np.newaxis]

    res += offset_mat
    return np.concatenate(res)


ic = init_incell(dia_prod, thk_i2o, ratio_slit, pitch_x, pitch_y)
print(ic.shape)
print(ic)

(33, 2)
[[-24.75   1.2 ]
 [-19.25   1.2 ]
 [-13.75   1.2 ]
 [ -8.25   1.2 ]
 [ -2.75   1.2 ]
 [  2.75   1.2 ]
 [  8.25   1.2 ]
 [ 13.75   1.2 ]
 [ 19.25   1.2 ]
 [ 24.75   1.2 ]
 [ 30.25   1.2 ]
 [-27.5    7.2 ]
 [-22.     7.2 ]
 [-16.5    7.2 ]
 [-11.     7.2 ]
 [ -5.5    7.2 ]
 [  0.     7.2 ]
 [  5.5    7.2 ]
 [ 11.     7.2 ]
 [ 16.5    7.2 ]
 [ 22.     7.2 ]
 [ 27.5    7.2 ]
 [-24.75  13.2 ]
 [-19.25  13.2 ]
 [-13.75  13.2 ]
 [ -8.25  13.2 ]
 [ -2.75  13.2 ]
 [  2.75  13.2 ]
 [  8.25  13.2 ]
 [ 13.75  13.2 ]
 [ 19.25  13.2 ]
 [ 24.75  13.2 ]
 [ 30.25  13.2 ]]


In [4]:
def duplicate_coords(
    coords: np.ndarray,
    ratio_slit: int,
    pitch_x: float,
    pitch_slit: float,
    lim_slit: float,
) -> np.ndarray:
    """セル座標をY方向に複製.

    Parameters
    ----------
    coords : np.ndarray
        _description_
    ratio_slit : int
        _description_
    pitch_x : float
        _description_
    pitch_slit : float
        _description_
    lim_slit : float
        _description_

    Returns
    -------
    np.ndarray
        Shape = (N, 2)
    """
    cnt = np.int64(np.ceil(lim_slit / pitch_slit))
    num = np.arange(-cnt, cnt + 1, 1)

    # スリット数分複製:Shape(スリット数, X個数, XY座標)
    res = np.tile(coords, (num.shape[0], 1, 1))

    # オフセット行列:Shape(スリット数, X個数, XY座標)
    offset_mat = np.zeros_like(num).astype(np.float64)
    offset_mat[1::2] += 0.5 * pitch_x if ratio_slit % 2 == 0 else 0.0
    offset_mat = np.column_stack([offset_mat, num * pitch_slit])

    res += offset_mat[:, np.newaxis]
    return np.concatenate(res)


cc = duplicate_coords(ic, ratio_slit, pitch_x, pitch_slit, lim_slit)
print(cc.shape)
print(cc)

(165, 2)
[[-24.75 -18.8 ]
 [-19.25 -18.8 ]
 [-13.75 -18.8 ]
 [ -8.25 -18.8 ]
 [ -2.75 -18.8 ]
 [  2.75 -18.8 ]
 [  8.25 -18.8 ]
 [ 13.75 -18.8 ]
 [ 19.25 -18.8 ]
 [ 24.75 -18.8 ]
 [ 30.25 -18.8 ]
 [-27.5  -12.8 ]
 [-22.   -12.8 ]
 [-16.5  -12.8 ]
 [-11.   -12.8 ]
 [ -5.5  -12.8 ]
 [  0.   -12.8 ]
 [  5.5  -12.8 ]
 [ 11.   -12.8 ]
 [ 16.5  -12.8 ]
 [ 22.   -12.8 ]
 [ 27.5  -12.8 ]
 [-24.75  -6.8 ]
 [-19.25  -6.8 ]
 [-13.75  -6.8 ]
 [ -8.25  -6.8 ]
 [ -2.75  -6.8 ]
 [  2.75  -6.8 ]
 [  8.25  -6.8 ]
 [ 13.75  -6.8 ]
 [ 19.25  -6.8 ]
 [ 24.75  -6.8 ]
 [ 30.25  -6.8 ]
 [-24.75  -8.8 ]
 [-19.25  -8.8 ]
 [-13.75  -8.8 ]
 [ -8.25  -8.8 ]
 [ -2.75  -8.8 ]
 [  2.75  -8.8 ]
 [  8.25  -8.8 ]
 [ 13.75  -8.8 ]
 [ 19.25  -8.8 ]
 [ 24.75  -8.8 ]
 [ 30.25  -8.8 ]
 [-27.5   -2.8 ]
 [-22.    -2.8 ]
 [-16.5   -2.8 ]
 [-11.    -2.8 ]
 [ -5.5   -2.8 ]
 [  0.    -2.8 ]
 [  5.5   -2.8 ]
 [ 11.    -2.8 ]
 [ 16.5   -2.8 ]
 [ 22.    -2.8 ]
 [ 27.5   -2.8 ]
 [-24.75   3.2 ]
 [-19.25   3.2 ]
 [-13.75   3.2 ]
 [ -8

In [ ]:
def offset_coords(
    coords: np.ndarray,
    pitch_x: float,
    pitch_slit: float,
    offset_x: bool,
    offset_y: bool,
) -> np.ndarray:
    """セル座標をオフセット.

    Parameters
    ----------
    coords : np.ndarray
        _description_
    pitch_x : float
        _description_
    pitch_slit : float
        _description_
    offset_x : bool
        _description_
    offset_y : bool
        _description_

    Returns
    -------
    np.ndarray
        Shape = (N, 2)
    """
    res = coords.copy()
    if offset_x:
        res[:, 0] += 0.5 * pitch_x
    if offset_y:
        res[:, 1] += 0.5 * pitch_slit
    return res


fc = offset_coords(cc, pitch_x, pitch_slit, True, True)
print(fc.shape)
print(fc)

(165, 2)
[[-22.   -13.8 ]
 [-16.5  -13.8 ]
 [-11.   -13.8 ]
 [ -5.5  -13.8 ]
 [  0.   -13.8 ]
 [  5.5  -13.8 ]
 [ 11.   -13.8 ]
 [ 16.5  -13.8 ]
 [ 22.   -13.8 ]
 [ 27.5  -13.8 ]
 [ 33.   -13.8 ]
 [-24.75  -7.8 ]
 [-19.25  -7.8 ]
 [-13.75  -7.8 ]
 [ -8.25  -7.8 ]
 [ -2.75  -7.8 ]
 [  2.75  -7.8 ]
 [  8.25  -7.8 ]
 [ 13.75  -7.8 ]
 [ 19.25  -7.8 ]
 [ 24.75  -7.8 ]
 [ 30.25  -7.8 ]
 [-22.    -1.8 ]
 [-16.5   -1.8 ]
 [-11.    -1.8 ]
 [ -5.5   -1.8 ]
 [  0.    -1.8 ]
 [  5.5   -1.8 ]
 [ 11.    -1.8 ]
 [ 16.5   -1.8 ]
 [ 22.    -1.8 ]
 [ 27.5   -1.8 ]
 [ 33.    -1.8 ]
 [-22.    -3.8 ]
 [-16.5   -3.8 ]
 [-11.    -3.8 ]
 [ -5.5   -3.8 ]
 [  0.    -3.8 ]
 [  5.5   -3.8 ]
 [ 11.    -3.8 ]
 [ 16.5   -3.8 ]
 [ 22.    -3.8 ]
 [ 27.5   -3.8 ]
 [ 33.    -3.8 ]
 [-24.75   2.2 ]
 [-19.25   2.2 ]
 [-13.75   2.2 ]
 [ -8.25   2.2 ]
 [ -2.75   2.2 ]
 [  2.75   2.2 ]
 [  8.25   2.2 ]
 [ 13.75   2.2 ]
 [ 19.25   2.2 ]
 [ 24.75   2.2 ]
 [ 30.25   2.2 ]
 [-22.     8.2 ]
 [-16.5    8.2 ]
 [-11.     8.2 ]
 [ -5